## <b>M6_감정 분석(Sentiment Analysis)</b>

### 1. 감정을 분석하는 방법

+ 감정 어휘 사전을 이용한 감정 상태 분류
  -  미리 분류해 둔 감정어 사전을 통해 분석하고자 하는 텍스트의 단어들을 사전에 기반해 분류하고, 그 감정 점수를 계산한다.
  - 이 때 사용되는 감정어 사전에는 해당 감정에 해당되는 단어를 미리 정의해둬야 한다.
+ 기계학습을 이용한 감정 상태 분류
  - 분석 데이터의 일부를 훈련 데이터로 사용해 그로부터 텍스트의 감정 상태를 분류한다.
  - 이 때 사용되는 훈련 데이터는 사용자가 분류한 감정 라벨이 포함되어 있어야 하며, 이를 인공 신경망, 의사 결정 트리 등의 기계 학습 알고리즘을 사용하여 분류한다.

### 2. afinn 감정 어휘 사전을 이용한 감정 분석

> 감정 사전 준비

*   감정 사전 라이브러리 afinn을 설치한다.
*   afinn은 영어에 대한 긍정, 부정에 대한 감정 사전을 제공한다.

> 데이터 준비

- 사용할 데이터를 구성 : 사이킷런에 내장되어 있는 뉴스그룹 데이터를 이용한다.

- 텍스트 파일 불러오기

In [ ]:
f = open('data_set/newsdata.txt', 'r', encoding='utf-8')
raw = f.read()
print(raw[:1000])

- 스트링을 리스트로 변환

In [ ]:
raw = eval(raw)
raw[:1]

> afinn 객체 생성

*   감정 사전을 구성하고 감정 스코어를 측정한다.
*   afinn 라이브러리는 감정 사전과 더불어 편리하게 감정가를 계산할 수 있는 함수를 제공한다.

In [ ]:
from afinn import Afinn

afinn = Afinn()

> 뉴스별 감정 점수 측정

> plt 스타일 설정

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('default')
plt.rc('font', family='Gulim', size=13)

> 시각화

In [ ]:
from tqdm import tqdm

positive = 0
neutral = 0
negative = 0

for i in tqdm(raw):
    score = afinn.score(i)
    if score > 0:
        positive += 1
    elif score == 0:
        neutral += 1
    else :
        negative += 1

In [ ]:
plt.figure(figsize=(10,5))
x_pos = np.array(['positive', 'neutral', 'negative'])
y_pos = [positive, neutral, negative]
plt.bar(x_pos, y_pos)
plt.show()

### 3. Scikit-learn을 이용한 기계학습 감정분석

> 데이터셋 파일 읽기
- 네이버 영화 리뷰 데이터 : https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt

In [ ]:
import pandas as pd

train_df = pd.read_csv('data_set/ratings.txt', sep = '\t', encoding = "utf-8")
train_df

- 샘플 데이터 선정 : 5만개

> 결측치 처리

> 내용과 평가를 Series 객체로 저장

> 데이터 셋 분리(Train용 80%, Test용 20%)

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(text, score , test_size=0.2, random_state=0)
print(len(train_x), len(train_y), len(test_x), len(test_y))

> TF-IDF 벡터화

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9)

tfidf.fit(train_x)

print(tfidf.get_feature_names_out()[:100])

In [ ]:
tfidf_train_x = tfidf.transform(train_x)
print(tfidf_train_x[:4])

> 감성 분석 모델 구축 : 로지스틱 회귀(Logistic Regression)

- 로지스틱 회귀(Logistic Regression) 분석
    - 일상 속 풀고자 하는 많은 문제 중에서는 두 개의 선택지 중에서 정답을 고르는 문제가 많다.
    - 감정분류에서는 긍정과 부정을 결정하는 문제에 활용한다.
    - 둘 중 하나를 결정하는 문제를 이진 분류(Binary Classification)라고 하며, 이런 문제를 풀기 위한 대표적인 알고리즘으로 로지스틱 회귀를 활용한다.

In [ ]:
from sklearn.linear_model import LogisticRegression


> 분석 모델 평가

- 평가 데이터 시각화

In [ ]:
test_df = pd.DataFrame({'text':test_x, 'label':test_y, 'predict':test_predict})
test_df

> 감정 분류